In [39]:
import torch
import torchvision.models as models
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
import pandas as pd

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pre-trained ResNet50
model = models.resnet50(weights='IMAGENET1K_V1')
model.eval()
model = model.to(device)

In [ ]:

from pathlib import Path
import os

path = Path("C:\\Users\\mcw\\Downloads\\imagenette2\\imagenette2\\val")
os.path.exists(path)


True

In [ ]:
# Define preprocessing
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load validation dataset
val_dataset = datasets.ImageFolder(root=path, transform=preprocess)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)

# Map ImageNet IDs to readable Imagenette class names
imagenette_labels = {
    'n01440764': 'tench', 'n02102040': 'English springer', 'n02979186': 'cassette player',
    'n03000684': 'chain saw', 'n03028079': 'church', 'n03394916': 'French horn',
    'n03417042': 'garbage truck', 'n03425413': 'gas pump', 'n03445777': 'golf ball',
    'n03888257': 'parachute'
}
class_names = val_dataset.classes
idx_to_readable_class = {i: imagenette_labels[c] for i, c in enumerate(class_names)}

In [41]:
#Perform inference
def predict_validation_set(model, val_loader):
    predictions = []
    true_labels = []
    image_paths = [p for p in val_dataset.imgs]
    
    model.eval()
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            probs = torch.nn.functional.softmax(outputs, dim=1)
            _, preds = torch.max(probs, dim=1)
            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
    
    return predictions, true_labels, image_paths

# Run inference
predictions, true_labels, image_paths = predict_validation_set(model, val_loader)

c:\Users\mcw\Desktop\jenkins-inference-pipeline\.venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


In [42]:
# Convert to readable class names
pred_classes = [idx_to_readable_class[p] for p in predictions]
true_classes = [idx_to_readable_class[t] for t in true_labels]
image_filenames = [p[0].split('/')[-1] for p in image_paths]

# Calculate and display accuracy
accuracy = accuracy_score(true_classes, pred_classes)
print(f"Top-1 Accuracy: {accuracy*100:.2f}%")

# Display sample predictions
print("\nSample Predictions:")
for i in range(5):
    print(f"Image: {image_filenames[i]}, Predicted: {pred_classes[i]}, True: {true_classes[i]}")

# Save results
results = pd.DataFrame({
    'Image': image_filenames,
    'Predicted': pred_classes,
    'True': true_classes
})
results.to_csv('imagenette_predictions.csv', index=False)
print("Predictions saved to imagenette_predictions.csv")

KeyError: np.int64(391)